## Referências.
- [Documentação oficial da API do Spotify](https://developer.spotify.com/documentation/web-api)
- [Documentação oficial do Spotipy, uma biblioteca python para acessar a API](https://spotipy.readthedocs.io/en/latest/#)
    - [Tutorial de primeiros passos com o Spotipy](https://medium.com/@maxtingle/getting-started-with-spotifys-api-spotipy-197c3dc6353b)

## Primeiros passos (ambiente linux)
1. Crie um ambiente virtual. Na pasta atual, rode no terminal: `make venv`
2. Ative o ambiente virtual: `source spot_env/bin/activate`
3. Instale as dependências: `pip install -r requirements.txt`
4. Crie uma pasta com as variáveis de ambiente:
    - `touch .env`
    - Configure as variáveis `SPOTIPY_CLIENT_ID` e `SPOTIPY_CLIENT_SECRET` com os valores disponíveis no app criado no [site do spotify para desenvolvedores](https://developer.spotify.com/dashboard). Não as compartilhe nem use-as diretamente em código público. 


In [4]:
# carrega as variáveis de ambiente
from dotenv import load_dotenv
load_dotenv()

import os
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import numpy as np

Credenciais da API

In [5]:
client_credentials_manager = SpotifyClientCredentials(
    client_id=os.environ.get('SPOTIPY_CLIENT_ID'),
    client_secret=os.environ.get('SPOTIPY_CLIENT_SECRET')
)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [81]:
playlists = {
    'Global': '37i9dQZEVXbNG2KDcFcKOF',
    'Brazil': '37i9dQZEVXbKzoK95AbRy9',
    'Australia': '37i9dQZEVXbK4fwx2r07XW',
    'Austria': '37i9dQZEVXbM1EaZ0igDlz',
    'Argentina': '37i9dQZEVXbKPTKrnFPD0G',
    'Chile': '37i9dQZEVXbLJ0paT1JkgZ',
    'Colombia': '37i9dQZEVXbL1Fl8vdBUba',
    'Mexico': '37i9dQZEVXbKUoIkUXteF6',
    'USA': '37i9dQZEVXbLp5XoPON0wI'
}

In [68]:
df = pd.read_csv('top_songs.csv')
set(df['playlist_id'])

{'37i9dQZEVXbK4fwx2r07XW',
 '37i9dQZEVXbKPTKrnFPD0G',
 '37i9dQZEVXbKUoIkUXteF6',
 '37i9dQZEVXbKzoK95AbRy9',
 '37i9dQZEVXbLp5XoPON0wI',
 '37i9dQZEVXbM1EaZ0igDlz',
 '37i9dQZEVXbNG2KDcFcKOF'}

### Criando o CSV das playlists

In [74]:
'''
dict: dicionário com o país e o id da sua respectiva playlist.
df: dataframe, caso pré-existente.
refresh: refaz as requisições para todas as playlists. False por padrão.
'''
def playlists_into_df(playlists_dict: dict, df: pd.DataFrame, refresh: bool = False) -> pd.DataFrame:
    country_list = []
    playlist_id_list = []
    artist_name = []
    track_name = []
    release_date = []
    popularity = []
    track_id = []
    for country, playlist_id in playlists_dict.items():
        # Pula requisição se ela já foi feita
        if (not refresh) and (playlist_id in set(df['playlist_id'])):
            continue
        # Faz uma requisição na API
        playlist = sp.playlist_items(
            playlist_id=playlist_id,
            fields='items.track(album.release_date, artists.name, name, popularity, id)'
        )
        tracks = playlist['items']
        for track in tracks:
            track = track['track']
            country_list.append(country)
            playlist_id_list.append(playlist_id)
            artist_name.append(track['artists'][0]['name'])
            track_name.append(track['name'])
            release_date.append(track['album']['release_date'])
            popularity.append(track['popularity'])
            track_id.append(track['id'])
        print(f'Added playlist from {country} with id {playlist_id}')
    new_df = pd.DataFrame({
        'playlist_country': country_list,
        'artist_name': artist_name,
        'track_name': track_name,
        'release_date': release_date,
        'popularity': popularity,
        'track_id': track_id,
        'playlist_id': playlist_id_list
    })
    if refresh:
        return new_df
    return pd.concat([df, new_df], ignore_index=True)

In [82]:
top_songs = playlists_into_df(playlists, df, False)

Added playlist from Chile with id 37i9dQZEVXbLJ0paT1JkgZ
Added playlist from Colombia with id 37i9dQZEVXbL1Fl8vdBUba


In [83]:
top_songs

,playlist_country,artist_name,track_name,release_date,popularity,track_id,playlist_id
0,Global,Artemas,i like the way you kiss me,2024-03-19,97,2GxrNKugF82CnoRFbQfzPf,37i9dQZEVXbNG2KDcFcKOF
1,Global,Hozier,Too Sweet,2024-03-22,86,0AjmK0Eai4zGrLaJwPvrDp,37i9dQZEVXbNG2KDcFcKOF
2,Global,Benson Boone,Beautiful Things,2024-04-05,82,3xkHsmpQCBMytMJNiDf3Ii,37i9dQZEVXbNG2KDcFcKOF
3,Global,Djo,End of Beginning,2022-09-16,99,3qhlB30KknSejmIvZZLjOD,37i9dQZEVXbNG2KDcFcKOF
4,Global,Ariana Grande,we can't be friends (wait for your love),2024-03-08,95,51ZQ1vr10ffzbwIjDCwqm4,37i9dQZEVXbNG2KDcFcKOF
...,...,...,...,...,...,...,...
445,Colombia,Tony Dize,Solos,2009-11-17,59,0WKd91LoIHCFIhDmgewjhy,37i9dQZEVXbL1Fl8vdBUba
446,Colombia,Dei V,Narcotics (with Bryant Myers),2023-09-14,81,0p0cOpBujR114Wirv5AM7W,37i9dQZEVXbL1Fl8vdBUba
447,Colombia,De La Ghetto,Sensacion Del Bloque,2006-01-01,78,5clFSlfkCRlhnH1cAQjSBi,37i9dQZEVXbL1Fl8vdBUba
448,Colombia,The Academy: Segunda Misión,"QUÍTENME EL TELÉFONO (feat. Yandel, Jay Wheeler)",2024-03-28,75,41XmmKJHx1ZAH0Iykgjxfx,37i9dQZEVXbL1Fl8vdBUba


In [84]:
# Salva o df em um csv
top_songs.to_csv('top_songs.csv', index=False)

### Criando o CSV com as features de cada música

Como a API só suporta requisições de 50 músicas, dividimo-as em batches.

In [13]:
def save_track_features_to_df(songs_df: pd.DataFrame) -> pd.DataFrame:
    df_list = []
    # Separa o df com todas as músicas em batches de 50 músicas
    batches = np.array_split(songs_df, len(songs_df) // 50)
    for batch in batches:
        track_features_list = sp.audio_features(tracks=batch['track_id']) # retorna uma lista de dicionários com as features
        for track_dict in track_features_list:
            df_list.append(pd.DataFrame([track_dict]))
    return pd.concat(df_list, ignore_index=True)

df = save_track_features_to_df(songs_df=pd.read_csv('top_songs.csv'))
df.to_csv('top_songs_features.csv', index=False)


/home/vinifm/spotify_hits_2/spot_venv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [14]:
playlist_df = pd.read_csv('top_songs.csv')
playlist_df.head(10)

,playlist_country,artist_name,track_name,release_date,popularity,track_id,playlist_id
0,Global,Artemas,i like the way you kiss me,2024-03-19,97,2GxrNKugF82CnoRFbQfzPf,37i9dQZEVXbNG2KDcFcKOF
1,Global,Hozier,Too Sweet,2024-03-22,86,0AjmK0Eai4zGrLaJwPvrDp,37i9dQZEVXbNG2KDcFcKOF
2,Global,Benson Boone,Beautiful Things,2024-04-05,82,3xkHsmpQCBMytMJNiDf3Ii,37i9dQZEVXbNG2KDcFcKOF
3,Global,Djo,End of Beginning,2022-09-16,99,3qhlB30KknSejmIvZZLjOD,37i9dQZEVXbNG2KDcFcKOF
4,Global,Ariana Grande,we can't be friends (wait for your love),2024-03-08,95,51ZQ1vr10ffzbwIjDCwqm4,37i9dQZEVXbNG2KDcFcKOF
5,Global,FloyyMenor,Gata Only,2024-02-02,96,6XjDF6nds4DE2BBbagZol6,37i9dQZEVXbNG2KDcFcKOF
6,Global,Future,Like That,2024-03-22,96,2tudvzsrR56uom6smgOcSf,37i9dQZEVXbNG2KDcFcKOF
7,Global,Tate McRae,greedy,2023-09-15,97,3rUGC1vUpkDG9CZFHMur1t,37i9dQZEVXbNG2KDcFcKOF
8,Global,Teddy Swims,Lose Control,2023-09-15,93,17phhZDn6oGtzMe56NuWvj,37i9dQZEVXbNG2KDcFcKOF
9,Global,Beyoncé,TEXAS HOLD 'EM,2024-03-29,87,7wLShogStyDeZvL0a6daN5,37i9dQZEVXbNG2KDcFcKOF


In [15]:
features_df = pd.read_csv('top_songs_features.csv')
features_df.head(10)

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.599,0.946,11,-4.263,1,0.0447,0.000938,0.010600,0.0826,0.747,151.647,audio_features,2GxrNKugF82CnoRFbQfzPf,spotify:track:2GxrNKugF82CnoRFbQfzPf,https://api.spotify.com/v1/tracks/2GxrNKugF82C...,https://api.spotify.com/v1/audio-analysis/2Gxr...,142515,4
1,0.741,0.620,10,-5.505,1,0.0412,0.029500,0.000809,0.0398,0.934,117.038,audio_features,0AjmK0Eai4zGrLaJwPvrDp,spotify:track:0AjmK0Eai4zGrLaJwPvrDp,https://api.spotify.com/v1/tracks/0AjmK0Eai4zG...,https://api.spotify.com/v1/audio-analysis/0Ajm...,251424,4
2,0.472,0.471,10,-5.692,1,0.0603,0.151000,0.000000,0.1400,0.219,105.029,audio_features,3xkHsmpQCBMytMJNiDf3Ii,spotify:track:3xkHsmpQCBMytMJNiDf3Ii,https://api.spotify.com/v1/tracks/3xkHsmpQCBMy...,https://api.spotify.com/v1/audio-analysis/3xkH...,180304,3
3,0.689,0.454,2,-7.643,1,0.0584,0.035100,0.002590,0.0707,0.912,159.982,audio_features,3qhlB30KknSejmIvZZLjOD,spotify:track:3qhlB30KknSejmIvZZLjOD,https://api.spotify.com/v1/tracks/3qhlB30KknSe...,https://api.spotify.com/v1/audio-analysis/3qhl...,159246,4
4,0.645,0.663,5,-8.305,1,0.0447,0.065700,0.000032,0.0751,0.287,115.830,audio_features,51ZQ1vr10ffzbwIjDCwqm4,spotify:track:51ZQ1vr10ffzbwIjDCwqm4,https://api.spotify.com/v1/tracks/51ZQ1vr10ffz...,https://api.spotify.com/v1/audio-analysis/51ZQ...,228639,4
5,0.791,0.499,8,-8.472,0,0.0509,0.446000,0.000024,0.0899,0.669,99.986,audio_features,6XjDF6nds4DE2BBbagZol6,spotify:track:6XjDF6nds4DE2BBbagZol6,https://api.spotify.com/v1/tracks/6XjDF6nds4DE...,https://api.spotify.com/v1/audio-analysis/6XjD...,222000,4
6,0.814,0.676,11,-4.670,0,0.2310,0.007090,0.000013,0.1190,0.312,162.012,audio_features,2tudvzsrR56uom6smgOcSf,spotify:track:2tudvzsrR56uom6smgOcSf,https://api.spotify.com/v1/tracks/2tudvzsrR56u...,https://api.spotify.com/v1/audio-analysis/2tud...,267707,4
7,0.750,0.733,6,-3.180,0,0.0319,0.256000,0.000000,0.1140,0.844,111.018,audio_features,3rUGC1vUpkDG9CZFHMur1t,spotify:track:3rUGC1vUpkDG9CZFHMur1t,https://api.spotify.com/v1/tracks/3rUGC1vUpkDG...,https://api.spotify.com/v1/audio-analysis/3rUG...,131872,1
8,0.561,0.604,9,-4.409,1,0.0337,0.199000,0.000019,0.1040,0.242,159.920,audio_features,17phhZDn6oGtzMe56NuWvj,spotify:track:17phhZDn6oGtzMe56NuWvj,https://api.spotify.com/v1/tracks/17phhZDn6oGt...,https://api.spotify.com/v1/audio-analysis/17ph...,210689,3
9,0.727,0.711,2,-6.549,1,0.0780,0.582000,0.000000,0.1580,0.375,110.012,audio_features,7wLShogStyDeZvL0a6daN5,spotify:track:7wLShogStyDeZvL0a6daN5,https://api.spotify.com/v1/tracks/7wLShogStyDe...,https://api.spotify.com/v1/audio-analysis/7wLS...,233457,4
